In [1]:
import numpy as np
import os, h5py
import pandas as pd
# from collections import OrderedDict
import matplotlib.pyplot as plt
from pyliftover import LiftOver
import subprocess

In [2]:
def fasta2list(fasta_file):
    fasta_coords = []
    seqs = []
    header = ''

    for line in open(fasta_file):
        if line[0] == '>':
            #header = line.split()[0][1:]
            fasta_coords.append(line[1:].rstrip())
        else:
            s = line.rstrip()
            s = s.upper()
            seqs.append(s)

    return fasta_coords, seqs

In [3]:
def dna_one_hot(seq):

    seq_len = len(seq)
    seq_start = 0
    seq = seq.upper()

    # map nt's to a matrix len(seq)x4 of 0's and 1's.
    seq_code = np.zeros((seq_len, 4), dtype='float16')

    for i in range(seq_len):
        if i >= seq_start and i - seq_start < len(seq):
            nt = seq[i - seq_start]
            if nt == 'A':
                seq_code[i, 0] = 1
            elif nt == 'C':
                seq_code[i, 1] = 1
            elif nt == 'G':
                seq_code[i, 2] = 1
            elif nt == 'T':
                seq_code[i, 3] = 1
            else:
                seq_code[i,:] = 0.25



    return seq_code

def enforce_const_range(site, window):
#     site -= 1 
    half_window = np.round(window/2).astype(int)
    start = site - half_window
    end = site + half_window
    return start, end



In [4]:
def expand_range(bedfile, output_filename, window=3072):
    df = pd.read_csv(bedfile, sep='\t', header=None, index_col=None)
    start, end = enforce_const_range(df.iloc[:,1].astype(int), window)
    df_expanded = df.copy()
    df_expanded.iloc[:,1] = start.values
    df_expanded.iloc[:,2] = end.values
    df_nonneg = df_expanded[df_expanded.iloc[:,1]>0]
    df_nonneg = df_nonneg.reset_index(drop=True)
    df_nonneg.to_csv(output_filename, header=None, sep='\t', index=None)

In [5]:
def combine_accessible_regions(basset_samplefile='/mnt/906427d6-fddf-41bf-9ec6-c3d0c37e766f/amber/ATAC/basset_sample_file.tsv',
                              output_accessible_regions='combined_atac.bed'):
    bed_paths = pd.read_csv(basset_samplefile, sep='\t', header=None)[1].values
    combined_csv = pd.concat([pd.read_csv(f, sep='\t', header=None) for f in bed_paths ])
    combined_csv.to_csv(output_accessible_regions, sep='\t', header=None, index=None)

In [6]:
def convert_hg19_to_hg38(bedfile, chain_file='/home/amber/QuantPred/datasets/VCF/hg19ToHg38.over.chain'):
    lo = LiftOver(chain_file)
    dataset = pd.read_csv(bedfile, header=None, sep='\t')
    hg38_coords = []
    for i, (chrom, start) in dataset.iloc[:,:2].iterrows():    
        result = lo.convert_coordinate(chrom, start)
        if len(result) == 1:
            hg38_coords.append(result[0][1])
        else:
            hg38_coords.append(None)
    dataset.iloc[:,1] = hg38_coords
    dataset.to_csv(bedfile, header=None, index=None, sep='\t')
    
def filter_accessible_vcfs(all_vcfs, output_bed,
                           current_genome='hg19',
                           chain_file='/home/amber/QuantPred/datasets/VCF/hg19ToHg38.over.chain',
                          accessible_regions = 'combined_accessibility.bed'):
    # if not provided make combined accessibility file
    if not os.path.isfile(accessible_regions):
        combine_accessible_regions(output_accessible_regions=accessible_regions)
    if current_genome == 'hg19': # convert coordinates to hg38
        convert_hg19_to_hg38(all_vcfs)
    dataset = pd.read_csv(all_vcfs, header=None, sep='\t')
#     dataset
    dataset.to_csv(all_vcfs, header=None, sep='\t', index=None)
    # intersect the hg38 coordinates with merged open chromatin region bed file for the current dataset
    cmd = "bedtools intersect -wa -a {} -b {} > vcf_open.bed;".format(all_vcfs, 
                                                                    accessible_regions)
    cmd += 'sort vcf_open.bed | uniq > {}'.format(output_bed)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    output, error = process.communicate()


In [11]:
def convert_bed_to_seq(bedfile, output_fa, genomefile='/home/shush/genomes/hg38.fa'): 
    cmd = 'bedtools getfasta -fi {} -bed {} -s -fo {}'.format(genomefile, bedfile, output_fa)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    coords_list, seqs_list = fasta2list(output_fa)
    return coords_list, seqs_list

def list_to_onehot(coords_list, seqs_list, nonneg_df):
    N = len(seqs_list)
    mid = window // 2
    onehot_ref = np.empty((N, window, 4))
    onehot_alt = np.empty((N, window, 4))
    coord_np = np.empty((N, 4)) # chrom, start, end coordinate array
    pos_dict = {'+': 1536, '-':1535}
    for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
        alt = ''
        strand = chr_s_e.split('(')[-1].split(')')[0]
        pos = pos_dict[strand]
        coord_np[i,3] = pos_dict[strand] - 1535

        assert seq[pos] == nonneg_df.iloc[i, 3], 'reference allele error'
        alt = nonneg_df.iloc[i,4]

        chrom, s_e = chr_s_e.split('(')[0].split(':')
        s, e = s_e.split('-')
        coord_np[i, :3] = int(chrom.split('chr')[-1]), int(s), int(e)

        onehot = dna_one_hot(seq)
        onehot_ref[i,:,:] = onehot

        onehot_alt[i, :, :] = onehot
        onehot_alt[i, mid, :] = dna_one_hot(alt)[0]

# dsQTL dataset

In [7]:
vcf_path = '.'
window = 3072


In [5]:
bed_paths = pd.read_csv('/mnt/906427d6-fddf-41bf-9ec6-c3d0c37e766f/amber/ATAC/basset_sample_file.tsv', sep='\t', header=None)[1].values
combined_csv = pd.concat([pd.read_csv(f, sep='\t', header=None) for f in bed_paths ])
combined_csv.to_csv('combined_atac.bed', sep='\t', header=None, index=None)

In [6]:
dsq_path = './dsQTL.csv'
dsq_all = pd.read_csv(dsq_path, sep='\t')

In [7]:
dsq_all

,chrom,snpChromStart,snpChromEnd,rsid,pred.fit.pctSig,strand,motifname,position,genotypes
0,chr1,13971,13972,"rs7536753,rs62530145,rs71259955",0.179,+,V_ETS_Q4,10,A/C
1,chr1,13979,13980,"rs2592732,rs62635296,rs151276478",0.109,-,V_AP1_C,1,G/A
2,chr1,14906,14907,"rs6682375,rs79585140",0.135,+,SPI1_ETS_1,11,A/G
3,chr1,14947,14948,rs201855936,0.110,-,V_AP2_Q6_01,3,T/C
4,chr1,16256,16257,"rs11489794,rs78588380",0.181,+,CTCF_UpstreamP1,30,C/G
...,...,...,...,...,...,...,...,...,...
483410,chrY,59027808,59027809,rs4047331,0.113,-,TEAD3_TEA_2,2,T/C
483411,chrY,59027861,59027862,rs76227494,0.104,+,V_TCF11MAFG_01,9,A/G
483412,chrY,59029164,59029165,rs79334518,0.151,-,V_CEBPB_02,10,G/A
483413,chrY,59029907,59029908,rs78084351,0.320,+,Jdp2.mouse_bZIP_1,5,A/C


In [8]:
dsq_filt = dsq_all[(dsq_all['chrom']!='chrX')&(dsq_all['chrom']!='chrY' )]
dsq_filt[['a1','a2']] = dsq_filt['genotypes'].str.split('/',expand=True) # write into separate columns
dsq_filt.to_csv('dsq_all.bed', sep='\t', header=False, index=None)

/home/amber/tf_2/lib/python3.7/site-packages/pandas/core/frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
!liftOver dsq_all.bed /home/amber/QuantPred/datasets/VCF/hg19ToHg38.over.chain dsq_hg38_all.bed unlifted.bed -bedPlus=3

Reading liftover chains
Mapping coordinates


In [10]:
!bedtools intersect -wa -a dsq_hg38_all.bed -b combined_atac.bed > dsq_open.bed

In [11]:
! sort dsq_open.bed | uniq > dsq_unique.bed

In [12]:
# import subprocess
# def bed_intersect(dataset_bed='dsq_all.bed', comb_peak='combined_atac.bed', out_path='dsq_open.bed'):
#     bashCmd = "bedtools intersect -a {} -b {} > {}".format(dataset_bed, comb_peak, out_path).split()
#     process = subprocess.Popen(bashCmd, stdout=subprocess.PIPE)
#     output, error = process.communicate()

# bed_intersect()

In [13]:
dsq_df = pd.read_csv('dsq_unique.bed', sep='\t', header=None, index_col=None)
dsq_df.columns = list(dsq_filt)
dsq_tmp = dsq_df[['chrom', 'snpChromStart', 'snpChromEnd', 'a1', 'a2', 'strand', 'rsid', 'pred.fit.pctSig']] # 
start, end = enforce_const_range(dsq_tmp['snpChromEnd']-1, window)
dsq_ext = dsq_tmp.copy()
dsq_ext.iloc[:,1] = start.values
dsq_ext.iloc[:,2] = end.values
dsq_nonneg = dsq_ext[dsq_ext['snpChromStart']>0]
dsq_nonneg = dsq_nonneg.reset_index(drop=True)
dsq_nonneg.to_csv('test_ds.csv', header=None, sep='\t', index=None)
!bedtools getfasta -fi /home/shush/genomes/hg38.fa -bed test_ds.csv -s -fo "test_ds.fa"

coords_list, seqs_list = fasta2list('test_ds.fa')

In [ ]:
cmd = 'bedtools getfasta -fi {} -bed {} -s -fo {}'.format(genomefile, bedfile, output_fa)
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
output, error = process.communicate()
coords_list, seqs_list = fasta2list(output_fa)

In [10]:
N = len(seqs_list)
mid = window // 2
onehot_ref = np.empty((N, window, 4))
onehot_alt = np.empty((N, window, 4))
coord_np = np.empty((N, 4)) # chrom, start, end coordinate array



for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos_dict = {'+': 1536, '-':1535}
    pos = pos_dict[strand]
    coord_np[i,3] = pos_dict[strand] - 1535
    
    if seq[pos] == dsq_nonneg['a1'][i]:
        alt = dsq_nonneg['a2'][i]
        
    elif seq[pos] == dsq_nonneg['a2'][i]:
        alt = dsq_nonneg['a1'][i]
    else:
        break
        print('reference allele error')

    chrom, s_e = chr_s_e.split('(')[0].split(':')
    s, e = s_e.split('-')
    coord_np[i, :3] = int(chrom.split('chr')[-1]), int(s), int(e)
    
    onehot = dna_one_hot(seq)
    onehot_ref[i,:,:] = onehot
    
    onehot_alt[i, :, :] = onehot
    onehot_alt[i, mid, :] = dna_one_hot(alt)[0]
        

NameError: name 'seqs_list' is not defined

In [15]:
onehot_ref.shape

(135306, 3072, 4)

In [16]:
np.array_equiv(onehot_ref, onehot_alt)

False

In [16]:
onehot_ref_alt = h5py.File(os.path.join(vcf_path, 'dsQTL_onehot.h5'), 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.create_dataset('fasta_coords', data=coord_np)
onehot_ref_alt.close()

# CAGI5 dataset

In [7]:
# read df and add strand
all_dfs = []
cagi_data = 'CAGI'
combined_filename = 'combined_cagi.bed'
for filename in os.listdir(cagi_data):
    prefix, regulator = filename.split('.tsv')[0].split('_')

    one_reg = pd.read_csv(os.path.join(cagi_data,filename), skiprows=7, sep='\t', header=None)
    one_reg['regulator'] = regulator
    one_reg['set'] = prefix
    all_dfs.append(one_reg)
    

combined_cagi = pd.concat(all_dfs)
combined_cagi.insert(4, 'strand', '+')
combined_cagi.insert(2,'end',combined_cagi.iloc[:,1]+1)
combined_cagi.iloc[:,0] = 'chr'+combined_cagi.iloc[:,0].astype(str)
combined_cagi.to_csv(combined_filename, sep='\t', header=False, index=None)

In [27]:
combined_cagi.iloc[4339,:]

0                chr10
1             51548987
end           51548988
2                    C
3                    A
strand               +
4                 0.02
5                 0.01
regulator         MSMB
set          challenge
Name: 0, dtype: object

In [8]:
# open_bedfile = 'cagi_open_vcf_unique.bed'
# filter_accessible_vcfs(combined_filename, open_bedfile, current_genome='xxx')

In [9]:
output_filename = 'nonneg_cagi_3K.bed'
expand_range(combined_filename, output_filename)

In [12]:
fa_filename = 'cagi_3k.fa'
coords_list, seqs_list = convert_bed_to_seq(output_filename, fa_filename, genomefile='/home/shush/genomes/hg19.fa')

In [13]:
nonneg_df[]


['chr11:5246732-5249804(+)',
 'chr11:5246732-5249804(+)',
 'chr11:5246732-5249804(+)',
 'chr11:5246733-5249805(+)',
 'chr11:5246733-5249805(+)',
 'chr11:5246733-5249805(+)',
 'chr11:5246734-5249806(+)',
 'chr11:5246734-5249806(+)',
 'chr11:5246734-5249806(+)',
 'chr11:5246735-5249807(+)',
 'chr11:5246735-5249807(+)',
 'chr11:5246735-5249807(+)',
 'chr11:5246736-5249808(+)',
 'chr11:5246736-5249808(+)',
 'chr11:5246736-5249808(+)',
 'chr11:5246737-5249809(+)',
 'chr11:5246737-5249809(+)',
 'chr11:5246737-5249809(+)',
 'chr11:5246738-5249810(+)',
 'chr11:5246738-5249810(+)',
 'chr11:5246738-5249810(+)',
 'chr11:5246739-5249811(+)',
 'chr11:5246739-5249811(+)',
 'chr11:5246739-5249811(+)',
 'chr11:5246740-5249812(+)',
 'chr11:5246740-5249812(+)',
 'chr11:5246740-5249812(+)',
 'chr11:5246741-5249813(+)',
 'chr11:5246741-5249813(+)',
 'chr11:5246741-5249813(+)',
 'chr11:5246742-5249814(+)',
 'chr11:5246742-5249814(+)',
 'chr11:5246742-5249814(+)',
 'chr11:5246743-5249815(+)',
 'chr11:524674

In [42]:
onehot.shape

(3072, 4)

In [56]:
window = 3072
bad_lines = []
N = len(seqs_list)
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None)
mid = window // 2
onehot_ref = []
onehot_alt = []
coord_np = np.empty((N, 4)) # chrom, start, end coordinate array
pos_dict = {'+': 1535, '-':1536}
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand]
#     coord_np[i,3] = pos_dict[strand] - 1535

    if seq[pos] != nonneg_df.iloc[i, 3]:
#         print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        alt = nonneg_df.iloc[i,4]

        onehot = dna_one_hot(seq)
        mutated_onehot = onehot
        mutated_onehot[pos] = dna_one_hot(alt)[0]
        onehot_ref.append(onehot)

        onehot_alt.append(mutated_onehot) 

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [58]:
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]

In [60]:
included_df.to_csv('final_cagi_metadata.csv')

In [62]:
onehot_ref_alt = h5py.File( 'CAGI_onehot.h5', 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.close()

# negative eQTL dataset

In [4]:
window = 3072
!grep negative ./eQTLs+negative.csv > negative.csv
!cut -d"," -f2- negative.csv  > negative_clean.csv

In [5]:
negative_table = pd.read_csv('negative_clean.csv', sep=',', skiprows=2,header =None,
                            names = ['chrom','end','a1','a2','probability','fold','label'])
negative_table['start'] = negative_table['end'] - 1
negative_table = negative_table[['chrom','start','end','a1','a2','probability','fold','label']]

In [6]:
negative_table['hg19start'] = negative_table['start']
negative_table['hg19end'] = negative_table['end']

In [7]:
negative_table

,chrom,start,end,a1,a2,probability,fold,label,hg19start,hg19end
0,chr1,10582,10583,G,A,0.35081,0,Random negative SNP,10582,10583
1,chr1,30922,30923,G,T,0.14216,0,Random negative SNP,30922,30923
2,chr1,52237,52238,T,G,0.18894,0,Random negative SNP,52237,52238
3,chr1,55163,55164,C,A,0.53187,0,Random negative SNP,55163,55164
4,chr1,88168,88169,C,T,0.48796,0,Random negative SNP,88168,88169
...,...,...,...,...,...,...,...,...,...,...
1091033,chrX,154891772,154891773,A,G,0.49647,9,360bp negative SNP,154891772,154891773
1091034,chrX,154900716,154900717,C,A,0.38994,9,360bp negative SNP,154900716,154900717
1091035,chrX,154913798,154913799,T,C,0.49817,9,360bp negative SNP,154913798,154913799
1091036,chrX,154925894,154925895,C,T,0.22989,9,360bp negative SNP,154925894,154925895


In [8]:
negative_table.to_csv('./negative_all.bed', sep='\t',header = None, index=None)

In [9]:
!liftOver negative_all.bed hg19ToHg38.over.chain negative_hg38_all.bed unlifted.bed -bedPlus=3

Reading liftover chains
Mapping coordinates


In [10]:
all_negative_table = pd.read_csv('negative_hg38_all.bed', sep='\t',header =None)
all_negative_table = all_negative_table[(all_negative_table[7]=='Random')]

In [11]:
all_negative_table

,0,1,2,3,4,5,6,7,8,9,10,11
0,chr1,10582,10583,G,A,0.35081,0,Random,negative,SNP,10582,10583
1,chr1,30922,30923,G,T,0.14216,0,Random,negative,SNP,30922,30923
2,chr1,52237,52238,T,G,0.18894,0,Random,negative,SNP,52237,52238
3,chr1,55163,55164,C,A,0.53187,0,Random,negative,SNP,55163,55164
4,chr1,88168,88169,C,T,0.48796,0,Random,negative,SNP,88168,88169
...,...,...,...,...,...,...,...,...,...,...,...,...
953976,chrX,156000144,156000145,A,T,0.67814,9,Random,negative,SNP,155229809,155229810
953977,chrX,156000160,156000161,C,T,0.62443,9,Random,negative,SNP,155229825,155229826
953978,chrX,156003432,156003433,T,C,0.45205,9,Random,negative,SNP,155233097,155233098
953979,chrX,156003449,156003450,G,T,0.36475,9,Random,negative,SNP,155233114,155233115


In [12]:
start, end = enforce_const_range(all_negative_table[2]-1,window)
all_negative_extended_ranges = pd.DataFrame({'chr':all_negative_table[0],'start':start, 'end':end,
                                    'a1':all_negative_table[3],'a2':all_negative_table[4],'hg19_start':all_negative_table[10] })
sample_negative_range = all_negative_extended_ranges.sample(n=10000,replace=False)
sample_negative_range.to_csv('negative_random_subsample_extended.csv', header=None, sep='\t', index=None)

In [13]:
sample_negative_range

,chr,start,end,a1,a2,hg19_start
633323,chr4,97394607,97397679,T,C,98317294
88271,chr10,49118957,49122029,G,A,50328538
37987,chr1,150305905,150308977,T,C,150279875
874439,chr8,111631211,111634283,A,G,112644976
352892,chr17,67855384,67858456,A,G,65853036
...,...,...,...,...,...,...
164153,chr12,751938,755010,T,G,862640
918953,chr9,119197356,119200428,G,T,121961170
761234,chr6,105889947,105893019,G,A,106339358
741506,chr6,40388990,40392062,C,T,40358265


In [15]:
!bedtools getfasta -fi /home/shush/genomes/hg38.fa -bed './negative_random_subsample_extended.csv' -s -fo 'negative_random_subsample_extended.fa'

In [16]:
coords, nucl_seqs = fasta2list('negative_random_subsample_extended.fa')

In [17]:
N = len(nucl_seqs)
mid = window // 2
onehot_ref = np.empty((N, window, 4))
onehot_alt = np.empty((N, window, 4))

for i in range(len(coords)):
    
    fa_allele = nucl_seqs[i][window//2]
    if fa_allele == sample_negative_range['a1'].values[i]:
        ref_allele = sample_negative_range['a1'].values[i]
        alt_allele = sample_negative_range['a2'].values[i]
    elif fa_allele == sample_negative_range['a2'].values[i]:
        ref_allele = sample_negative_range['a2'].values[i]
        alt_allele = sample_negative_range['a1'].values[i]
    else:
        print('reference allele error' + str(i))
        next
#     ref_allele = chr8_random_negative_table[3].values[i]
#     alt_allele = chr8_random_negative_table[4].values[i]
#     assert nucl_seqs[i][window//2] == ref_allele, str(i) + ' reference allele error'

    onehot = dna_one_hot(nucl_seqs[i])
    onehot_ref[i,:,:] = onehot
    
    onehot_alt[i, :, :] = onehot
    onehot_alt[i, mid, :] = dna_one_hot(alt_allele)[0]


coord_np = np.array([[8 for i in range(len(coords))], start.values, end.values]).T



reference allele error362
reference allele error538
reference allele error1532
reference allele error1731
reference allele error2746
reference allele error2842
reference allele error3844
reference allele error4079
reference allele error4447
reference allele error4714
reference allele error6277
reference allele error6423
reference allele error6495
reference allele error8444
reference allele error8758
reference allele error9931


/home/amber/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [18]:
coords[362]

'chr6:61283848-61286920(61952329)'

In [19]:
sample_negative_range.iloc[362]

chr               chr6
start         61283848
end           61286920
a1                   G
a2                   T
hg19_start    61952329
Name: 747273, dtype: object

In [21]:
nucl_seqs[362][window//2]

'A'

In [29]:
sample_negative_range

,chr,start,end,a1,a2,hg19_start
118697,chr11,3186192,3189264,A,G,3208958
795919,chr7,29543878,29546950,C,T,29585030
169707,chr12,12971032,12974104,G,C,13125502
286729,chr15,62983620,62986692,T,C,63277355
567090,chr3,100898392,100901464,T,C,100618772
...,...,...,...,...,...,...
437752,chr2,73254392,73257464,T,C,73483056
313391,chr16,29116523,29119595,A,C,29129380
265083,chr14,89493650,89496722,G,A,89961530
777585,chr6,158224817,158227889,C,A,158647385


In [29]:
vcf_path = './'
onehot_ref_alt = h5py.File(os.path.join(vcf_path, 'negative_onehot.h5'), 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.create_dataset('fasta_coords', data=coord_np)
onehot_ref_alt.close()

In [87]:
import numpy as np

# Try background seq generation in hg19

In [43]:
window = 3072
!grep negative ./eQTLs+negative.csv > negative.csv
!cut -d"," -f2- negative.csv  > negative_clean.csv
negative_table = pd.read_csv('negative_clean.csv', sep=',', skiprows=2,header =None,
                            names = ['chrom','end','a1','a2','probability','fold','label'])
negative_table['start'] = negative_table['end'] - 1
negative_table = negative_table[['chrom','start','end','a1','a2','probability','fold','label']]

In [44]:
all_negative_table = negative_table[(negative_table['label'] == 'Random negative SNP')&
                                    (negative_table['chrom']!='chrX')&(negative_table['chrom']!='chrY')]
start, end = enforce_const_range(all_negative_table['end']-1,window)
all_negative_extended_ranges = pd.DataFrame({'chr':all_negative_table['chrom'],'start':start, 'end':end,
                                    'a1':all_negative_table['a1'],'a2':all_negative_table['a2']})
sample_negative_range = all_negative_extended_ranges.sample(n=10000,replace=False)
sample_negative_range.to_csv('negative_random_subsample_extended.csv', header=None, sep='\t', index=None)

In [45]:
!bedtools getfasta -fi /home/shush/genomes/hg19.fa -bed './negative_random_subsample_extended.csv' -s -fo 'negative_random_subsample_extended.fa'

In [46]:
coords, nucl_seqs = fasta2list('negative_random_subsample_extended.fa')

In [48]:
coords[0]

'chr9:24623578-24626650()'

In [51]:
N = len(nucl_seqs)
mid = window // 2
onehot_ref = np.empty((N, window, 4))
onehot_alt = np.empty((N, window, 4))
coord_np = np.empty((N, 4))

for i in range(len(coords)):
    
    fa_allele = nucl_seqs[i][window//2]
    if fa_allele == sample_negative_range['a1'].values[i]:
        ref_allele = sample_negative_range['a1'].values[i]
        alt_allele = sample_negative_range['a2'].values[i]
    elif fa_allele == sample_negative_range['a2'].values[i]:
        ref_allele = sample_negative_range['a2'].values[i]
        alt_allele = sample_negative_range['a1'].values[i]
    else:
        print('reference allele error' + str(i))
        next
#     ref_allele = chr8_random_negative_table[3].values[i]
#     alt_allele = chr8_random_negative_table[4].values[i]
#     assert nucl_seqs[i][window//2] == ref_allele, str(i) + ' reference allele error'

    onehot = dna_one_hot(nucl_seqs[i])
    onehot_ref[i,:,:] = onehot
    
    onehot_alt[i, :, :] = onehot
    onehot_alt[i, mid, :] = dna_one_hot(alt_allele)[0]

    
    chr_s_e = coords[i]
    chrom, s_e = chr_s_e.split('(')[0].split(':')
    s, e = s_e.split('-')
    coord_np[i, :3] = int(chrom.split('chr')[-1]), int(s), int(e)




In [54]:
vcf_path = './'
onehot_ref_alt = h5py.File(os.path.join(vcf_path, 'negative_onehot.h5'), 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.create_dataset('fasta_coords', data=coord_np)
onehot_ref_alt.close()

In [53]:
onehot_ref_alt.close()